Clima usando visualcrossing

In [4]:
import pandas as pd
import requests
from datetime import datetime
import time

API_KEY = "8YS5FQZNC8N6DGE6X2AY56Z3Z"

def format_datetime(data, hora):
    try:
        dt = datetime.strptime(f"{data} {hora}", "%d/%m/%Y %H:%M")
        return dt.strftime("%Y-%m-%dT%H:%M:00")
    except:
        return None

def get_weather(cidade, data_iso):
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{cidade}/{data_iso}?key={API_KEY}&unitGroup=metric&include=hours"

    try:
        r = requests.get(url)
        data = r.json()

        # Tenta localizar a hora exata
        hora = datetime.fromisoformat(data_iso).hour
        hora_dados = next((h for h in data['days'][0]['hours'] if h['datetime'].startswith(f"{hora:02d}")), None)

        if hora_dados:
            return {
                "Temperatura (°C)": hora_dados.get("temp"),
                "Umidade (%)": hora_dados.get("humidity"),
                "Precipitação (mm)": hora_dados.get("precip", 0),
                "Vento (km/h)": hora_dados.get("windspeed", 0)
            }
        else:
            return None
    except Exception as e:
        print(f"Erro buscando clima para {cidade} {data_iso}: {e}")
        return None

def main():
    df = pd.read_csv("cidades_partidas_corrigido.csv")

    clima_list = []

    for i, row in df.iterrows():
        cidade = row["cidade"]
        data = row["data"]
        hora = row["hora"]

        data_iso = format_datetime(data, hora)
        if not data_iso:
            print(f"Data inválida: {data} {hora}")
            clima_list.append({})
            continue

        print(f"🌦️ Buscando clima de {cidade} em {data_iso}")
        clima = get_weather(cidade, data_iso)
        clima_list.append(clima if clima else {})

        time.sleep(1.1)  # evitar estourar a cota gratuita

    clima_df = pd.DataFrame(clima_list)
    df_final = pd.concat([df, clima_df], axis=1)
    df_final.to_csv("jogos_com_clima_2024.csv", index=False, encoding='utf-8-sig')
    print("✅ Arquivo salvo como jogos_com_clima_2024.csv")

if __name__ == "__main__":
    main()


🌦️ Buscando clima de Salvador em 2024-04-14T18:30:00
Erro buscando clima para Salvador 2024-04-14T18:30:00: Expecting value: line 1 column 1 (char 0)
🌦️ Buscando clima de Criciuma em 2024-04-13T18:30:00
Erro buscando clima para Criciuma 2024-04-13T18:30:00: Expecting value: line 1 column 1 (char 0)
🌦️ Buscando clima de Porto Alegre em 2024-04-13T18:30:00
Erro buscando clima para Porto Alegre 2024-04-13T18:30:00: Expecting value: line 1 column 1 (char 0)


KeyboardInterrupt: 